In [104]:
import numpy as np
import random
import pandas as pd

In [105]:
students = pd.read_csv('../dataset_full.csv')

# hyperparameters
num_individuals = 50
groupsize = 5

Darwinian Natural Selection

- Variation
- Selection
- Heredity

1. Variation (Create a population)

In [106]:
students.head()

,ID,Name,Gender,Preferred language,Majors,Level of ambition,Prefered meeting place,Personality type,Best friend,Preferred day
0,1,Karolin Traugott,Female,Any,"('CL', 'AI')",Low,In person,ENTJ,Lukas Ostermann,Friday
1,2,Jana Kastner,Female,Any,"('PHIL', 'CL')",Medium,Online,ENTJ,Jan Gaertner,Wednesday
2,3,Leah Berg,Female,Any,"('PHIL', 'CP')",Very high,In person,ENTP,Antje Sommer,Friday
3,4,Marko Diederich,Male,Any,"('AI', 'PHIL')",High,In person,INTJ,Jens Beyer,Monday
4,5,Franziska Pfeiffer,Female,English,"('NI', 'CP')",Low,In person,INFJ,Ralph Kluge,Monday


In [107]:
class Population():
    def __init__(self, students, n_individuals, groupsize):
        super(Population, self).__init__()

        if n_individuals % groupsize != 0:
            raise ValueError(f'Received unexpected group size: {n_individuals} groups % {groupsize} individuals has to be 0.')
        
        self.students = students
        self.n_individuals = n_individuals
        self.ids = students.ID.tolist()
        self.groupsize = groupsize

    def create_random_individual(self):
        individual = self.ids.copy()
        random.shuffle(individual)

        return individual

    def create_initial_population(self):
        population = []
        for _ in range(self.n_individuals):
            population.append(self.create_random_individual())

        return np.array(population, dtype=int)

    def show_groups_from_individual(self, individual):

        groups = []
        for i in individual:
            groups.append(self.students['Name'][i-1])
        
        for n,j in enumerate(range(0, len(groups), self.groupsize)):
            print(f'Group {n+1} :',groups[j:j+5])

        return None

In [108]:
population = Population(students, num_individuals, groupsize)

test_population = population.create_initial_population()

test_population

array([[ 40,  51,  96, ...,  55,  74,  99],
       [ 66,  54,  11, ...,  47,   5,  95],
       [ 42,  89, 100, ...,  41,  94,  87],
       ...,
       [ 10,  93,  95, ...,  64,   2,  78],
       [ 45,  81,   4, ...,  15,  82,  69],
       [ 64,  89,  24, ...,  94,  86,  25]])

Show Groups from the IDs

In [109]:
population.show_groups_from_individual(test_population[0])

Group 1 : ['Tim Ritter', 'Karin Schultz', 'Kathrin Nagel', 'Petra Konig', 'Marko Diederich']
Group 2 : ['Maik Egger', 'Tobias Hueber', 'Anna Fuhrmann', 'Uta Fruehauf', 'Vanessa Lehrer']
Group 3 : ['Ralph Weber', 'Andreas Fassbinder', 'Ralph Kluge', 'Simone Köhler', 'Jens Engel']
Group 4 : ['Katja Fuerst', 'Christina Fischer', 'Jan Gaertner', 'Jörg Amsel', 'Thomas Faust']
Group 5 : ['Mandy Neustadt', 'Leah Berg', 'Niklas Walter', 'Maximilian Baecker', 'Paul Schwartz']
Group 6 : ['Kathrin Schwab', 'Jens Unger', 'Martina Wolf', 'Tanja Probst', 'Ines Lemann']
Group 7 : ['Kathrin Weisz', 'Sven Gloeckner', 'Dennis Fuerst', 'Phillipp Fleischer', 'Lea Goldschmidt']
Group 8 : ['Klaus Gerber', 'Lukas Ostermann', 'Antje Eichelberger', 'Leonie Hueber', 'Johanna Schroder']
Group 9 : ['Daniela Schulz', 'Bernd Pabst', 'Laura Achen', 'Brigitte Schiffer', 'Sandra Reiniger']
Group 10 : ['Stephan Lehmann', 'Christin Schmidt', 'Daniel Fischer', 'Kathrin Fenstermacher', 'Susanne König']
Group 11 : ['Tobias

________

2. Selection (Evaluate the fitness of each group, find fittest)

In [110]:
class Fitness():
    def __init__(self):
        super(Fitness, self).__init__()

    def evaluate_fitness(self, individual, students):
        """ Evaluates the fitness of an individual. """

        # split individual into student groups of the groupsize
        groups = np.array_split(individual, (len(individual)/groupsize))

        # iterate over groups and calculate scores for the different parameters
        scores = []
        for group_ids in groups:
            # get full data for students in this group from pd dataframe
            group = students.loc[students['ID'].isin(group_ids)]

            # get individual scores for parameters
            language_score = self.evaluate_language(group)
            if language_score == -1:
                # if we really want to use a hard constraint we would need to return -1 here, this makes for really bad (initial) results though
                # return -1
                scores.append(language_score)
                continue

            major_score = self.evaluate_majors(group)
            ambition_score = self.evaluate_ambition(group)
            place_score = self.evaluate_meeting_place(group)
            gender_score = self.evaluate_gender(group)
            friend_score = self.evaluate_friends(group)
            personality_score = self.evaluate_personality(group)
            day_score = self.evaluate_meeting_day(group)

            # formula for adding and weighting different scores
            scores.append(language_score+major_score+ambition_score+place_score+gender_score+friend_score+personality_score+day_score)

        #Convert to series to calculate mean more easily
        return pd.Series(scores).mean()

    def evaluate_language(self, group):
        # number of groupmembers per language
        counts = group['Preferred language'].value_counts()

        # hard constraints, if languages are conflicting, return -1
        if 'German' in counts.index and 'English' in counts.index:
            return -1

        return groupsize

    def evaluate_majors(self, group):
        majors = group['Majors'].tolist()

        # preprocess majors from dataset notation to list with all majors
        group_majors = []
        for pair in majors:
            pair = pair[1:-1].split(", ")

            group_majors.append(pair[0][1:-1])
            group_majors.append(pair[1][1:-1])

        #convert to Series for easier handling
        group_majors = pd.Series(group_majors)
        #get value counts
        group_major_values = group_majors.value_counts()
        #remove majors only one person takes (as they provide no synergy to the group)
        group_major_values = group_major_values[group_major_values > 1]

        #add number of shared majors and divide by 2; Formula is kinda arbitrary
        return group_major_values.sum() /2

    def evaluate_ambition(self, group):
        # get pd Series of ambitions
        ambitions = group['Level of ambition']
        # get int value mappings for ambitions

        mapping = {
            'Very low': 1,
            'Low': 2,
            'Medium': 3,
            'High': 4,
            'Very high': 5
        } 

        ambitions = ambitions.map(mapping)

        # fitness is groupsize - variance in group motivation (so less variance = more fitness)
        return groupsize - ambitions.var()

    def evaluate_meeting_place(self, group):
        # number of groupmembers for each preferred meeting place
        meeting_place = group['Prefered meeting place'].value_counts()

        # if all prefer the same meeting place return 5, else 0
        if meeting_place[0] == groupsize:
            return 5

        return 0

    def evaluate_gender(self, group):
        # evaluate by variance
        genders = group['Gender'].value_counts()

        # add 0 entry for missing genders
        for gender in ['Male', 'Female', 'Indeterminate']:
            if gender not in genders.index:
                genders[gender] = 0

        # return groupsize - variance
        return groupsize - genders.var()

    def evaluate_friends(self, group):
        #for each member +1 if friend is also in group
        group_member_names = group['Name'].tolist()
        best_friends_name = group['Best friend'].tolist()

        # get intersection between both lists
        friends_in_group = list(set(group_member_names).intersection(best_friends_name))

        # fitness += 1 for every pair of friends
        return len(friends_in_group)

    def evaluate_personality(self, group):
        #information about compatible personality types is taken from
        # Montequín, Vicente Rodríguez, et al. "Using Myers-Briggs type indicator (MBTI) as a tool for setting up student teams for information technology projects." Journal of Information Technology and Application in Education 1.1 (2012): 28-34.

        #count existing personality types in each group
        personalities = group['Personality type']
        types = personalities.value_counts()

        #fitness function starts with 0 and gets better
        # with every good group member
        fitness = 0

        #its good if there is a group leader like an ISTJ or an ESTJ, but only one
        try:
            if (types['ISTJ'] + types['ESTJ'] == 1):
                fitness+=5
            elif (types['ISTJ'] + types['ESTJ'] >= 2):
                fitness-=5
        except KeyError:
            pass

        #compare compatibility of group members
        for i, personality_a in enumerate(personalities.tolist()):
            for j, personality_b in enumerate(personalities.tolist()):
                # skip same group member and members already compared
                if i <= j:
                    continue

                # increase fitness if
                if (personality_a[1] != personality_b[1]) ^ (personality_a[2] != personality_b[2]):
                    if (personality_a[0] != personality_b[0]) or (personality_a[3] != personality_b[3]):
                        fitness+=1

        return fitness

    def evaluate_meeting_day(self, group):
        # number of groupmembers for each preferred meeting day
        meeting_day = group['Preferred day'].value_counts()

        # if all prefer the same meeting day return 5, else 0
        if meeting_day[0] == groupsize:
            return 5

        return 0

    def mean_fitness(self, population):
        # get list of fitness scores for all individuals in this population
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        # convert to series to calculate mean more easily
        return round(pd.Series(fitness_scores).mean(), 3)

    def best_fitness(self, population):
        # get list of fitness scores for all individuals in this population
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        # sort by best first
        scores_sorted = sorted(fitness_scores, reverse=True)

        # return fitness of best individual
        return round(scores_sorted[0], 3)

    def indices_sorted_by_fitness(self, population):
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        indices = np.argsort(fitness_scores)

        return indices.tolist()

In [111]:
best_fitness = Fitness().best_fitness(test_population)
best_fitness

20.292

3. Heredity (Pass on the fittest to the next generation)

In [118]:
class Crossover():
    def __init__(self):
        super(Crossover, self).__init__()

    def uniform_order_crossover(self, p1, p2, population):

        self.population = population

        template = self.get_crossover_template(population, crossover_rate=0.2)
        # create 'empty' child
        child = np.zeros((len(p1),),dtype=int)
        # where the template is true, take values from p1
        child[template] = p1[template]
        # store genes used from p1
        used_genes = p1[template]

        # get all genes from p2
        remaining_genes = p2.tolist()
        # add genes from p2 (that were not used from p1) to the empty spots of the child
        for i, value in enumerate(child):
            # if this spot is already filled, continue
            if value != 0:
                continue

            # do while:  pop(get and remove) next gene from p2 until one is found that is not yet in the genome of the child, then add that
            while True:
                next_gene = remaining_genes.pop(0)
                if next_gene not in used_genes:
                    child[i] = next_gene
                    break

        return child

    def get_crossover_template(self,pop, crossover_rate = 0.2):
        # initialize template with false values
        template = np.zeros((len(pop[0]),),dtype=bool)
        # get random indices of the amount #of genes * crossover rate
        random_indices = np.random.choice(template.shape[0], int(len(pop[0])*crossover_rate), replace=False)
        #set these indices to true
        template[random_indices] = True

        return template

    def tournament_selection(self, popuplation, tournament_size = 8):
        # get random indices to select random individuals from population
        random_indices = np.random.choice(popuplation.shape[0], tournament_size*2, replace=False)

        # get individuals from random indices and split into two tournaments
        tournament1 = popuplation[random_indices[:tournament_size]]
        tournament2 = popuplation[random_indices[tournament_size:]]

        parents = []
        # tournament is won by fittest individual in each tournament, those become the two parents
        for tournament in (tournament1,tournament2):
            # get fitness scores for every individual in the tournament
            fitness_scores = [Fitness().evaluate_fitness(individual, students) for individual in tournament]
            # get indices ordered by highest fitness first
            idx = np.argsort(fitness_scores)[::-1]
            # add individual with highest fitness to parents
            parents.append(tournament[idx[0]])

        return parents

In [120]:
random_population = Population(students, num_individuals, groupsize).create_initial_population()

print(random_population, '\n')

random_parent1, random_parent2 = Crossover().tournament_selection(random_population)

print(random_parent1, '\n')
print(random_parent2, '\n')

child = Crossover().uniform_order_crossover(random_parent1, random_parent2, random_population)

print(child)

[[ 29  88  37 ...  25  30  61]
 [ 96  43  82 ...  69  54  41]
 [ 86  77  55 ... 100  57   5]
 ...
 [ 49   8  60 ...   4  11   9]
 [  3  27  84 ...  47  11  15]
 [ 11  23   3 ...  54  10  18]] 

[ 99   2  45  85  15  92  40  60   3  81  74  42  57  16  78   8  41  26
  62   4  66   6  59  53  64  68  20  52   1  54  25  37  86  34  12  79
  55  29  91  65  36  82  71  43  18  28  14  47  33  58  19  73  83  48
  32  39  93  95  75  80  50  24  98  10  22  76  89  61   9   7  90  72
  44  51  69  13  23  87  11  38  97  31  67  30  21  70  56  84  49  88
   5  63  77 100  17  27  94  96  46  35] 

[ 96  43  82  51  99   6  56  66  13  55  89  58  81  25  48  73  28  23
  60  20   2   5  77  86  87  68  34  63   3  61  57  16  97  38  79  95
  22  31  94  90   7  33  50  11  98  62  26  18  32  36  85  46  83  14
  59  49  40  45  44  52  27   4  91  39  19   8   9  47  17  67  93  92
   1  75  29  71  88  42  10  76  72  53  65  80  12  35  64  30  78  74
  15  21  37  84 100  24  70  69

We also add a mutation function: 

In [121]:
def mutation(individual, mutation_rate = 0.2):
    # if random percentage is lower than the mutation rate, switch two random genes
    if random.uniform(0, 1) < mutation_rate:
        idx1 = random.randint(0, len(individual)-1)
        idx2 = random.randint(0, len(individual)-1)

        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

    return individual

Finally, our executable GA class:

In [122]:
class Genetic_Algorithm():
    def __init__(self, students, num_individuals, groupsize):
        super(Genetic_Algorithm, self).__init__()

        self.students = students
        self.groupsize = groupsize
        self.num_individuals = num_individuals
        self.ids = students.ID.tolist()

    def run(self, episodes, replace='all', mutation_rate=0.2):

        self.epidodes = episodes
        self.population = Population(students, num_individuals, groupsize).create_initial_population()

        print("episode " + str(0) + ": mean fitness score: " + str(Fitness().mean_fitness(self.population)) + "; best individual fitness: " + str(Fitness().best_fitness(self.population)))

        if replace == 'all':
            for episode in range(episodes):
                new_pop = []
                # we get two new individuals by each step, so half the pop size
                for _ in range(len(self.population)//2):
                    # find two parents by tournament selection
                    p1, p2 = Crossover().tournament_selection(self.population,8)

                    # get random binary template for crossover
                    crossover_template = Crossover().get_crossover_template(self.population, crossover_rate = 0.2)

                    # create two children by uniform order crossover
                    c1 = Crossover().uniform_order_crossover(p1,p2,crossover_template)
                    c2 = Crossover().uniform_order_crossover(p2,p1,crossover_template)
                    # do mutation
                    c1 = mutation(c1, mutation_rate)
                    c2 = mutation(c2, mutation_rate)
                    # add children to new population
                    new_pop.append(c1)
                    new_pop.append(c2)

                pop = np.array(new_pop,dtype=int)

                print("episode " + str(episode+1) + ": mean fitness score: " + str(Fitness().mean_fitness(pop)) + "; best individual fitness: " + str(Fitness().best_fitness(pop)))

        elif list(replace):
            for episode in range(episodes):
                pop_indices_sorted_by_fitness = Fitness().indices_sorted_by_fitness(self.population)
                
                # we get two new individuals by each step, so half the pop size
                for _ in range(replace//2):
                    # find two parents by tournament selection
                    p1, p2 = Crossover().tournament_selection(self.population,8)

                    # get random binary template for crossover
                    crossover_template = Crossover().get_crossover_template(self.population, crossover_rate = 0.2)

                    # create two children by uniform order crossover
                    c1 = Crossover().uniform_order_crossover(p1,p2,crossover_template)
                    c2 = Crossover().uniform_order_crossover(p2,p1,crossover_template)
                    # do mutation
                    c1 = mutation(c1)
                    c2 = mutation(c2)
                    # add children to new population
                    next_idx = pop_indices_sorted_by_fitness.pop(0)
                    pop[next_idx] = c1
                    next_idx = pop_indices_sorted_by_fitness.pop(0)
                    pop[next_idx] = c2

                print("episode " + str(episode+1) + ": mean fitness score: " + str(Fitness().mean_fitness(pop)) + "; best individual fitness: " + str(Fitness().best_fitness(pop)))

In [123]:
Genetic_Algorithm(students, num_individuals, groupsize).run(episodes=10, replace='all', mutation_rate=0.2)

episode 0: mean fitness score: 16.624; best individual fitness: 20.102


TypeError: object of type 'numpy.bool_' has no len()